In [1]:
import pandas as pd
import numpy as np
import datetime
import warnings
warnings.filterwarnings('ignore')
np.set_printoptions(suppress=True)# arry 数字 不用科学计数法

In [ ]:
#-----------------------收集2021年每天每小时系统借车总量（以后不必运行）-------------------------------------
list0 = ["00","01","02","03","04","05","06","07","08","09","10","11","12","13","14","15","16","17","18","19","20","21","22","23","24","25","26","27","28","29","30","31"]
num_per_hour = {}
for month in list0[1:13]:
    temp = pd.read_csv("‪E:/CitiBike/ori_data/2021"+ month +"-citibike-tripdata.csv")['ended_at']   #还车
    #temp = pd.read_csv("E:/CitiBike/ori_data/2021"+ month +"-citibike-tripdata.csv")['started_at']#借车
    for n in range(len(temp)):
        temp[n] = temp[n][8:10]+temp[n][11:13]
    if month in ['01','03','05','07','08','10','12']:
        list1 = list0[1:32]
    elif month in ['04','06','09','11']:
        list1 = list0[1:31]
    else:
        list1 = list0[1:29]
    for day in list1:
        for h in list0[:24]:
            num_per_hour['2021'+'-'+month+'-'+day+' '+h]=len(temp[temp==day+h])
#(pd.DataFrame([num_per_hour],index=['pick_num']).T).to_csv('E:/CitiBike/y0_output/total_pick_per_hour.csv')     #借车
(pd.DataFrame([num_per_hour],index=['drop_num']).T).to_csv('E:/CitiBike/y0_output/total_drop_per_hour.csv')     #还车

In [ ]:
#-----------------------爬取站点库存状态数据-------------------------------

In [ ]:
import sys
import urllib.request
import json
from time import gmtime, strftime
import datetime, pytz

def length0(z):
    if len(str(z))==1:
        return "0"+str(z)
    else:
        return str(z)
    
status = urllib.request.urlopen("https://gbfs.citibikenyc.com/gbfs/en/station_status.json").read()
a = datetime.datetime.now(pytz.timezone('US/Eastern'))
str_ = status.decode('UTF-8')
dict_ = json.loads(str_)
dtation_info_df = pd.DataFrame(dict_['data']['stations'])
#dtation_info_df[['legacy_id','short_name']].to_csv('E:/CitiBike/y0_output/new_old_name.csv') #记得累计更新
dtation_info_df = dtation_info_df[['station_id','num_ebikes_available','num_docks_available','num_bikes_available','station_status','num_docks_disabled','num_bikes_disabled']]
dtation_info_df['capacity'] = dtation_info_df['num_bikes_available']+dtation_info_df['num_docks_available']+dtation_info_df['num_docks_disabled']+dtation_info_df['num_bikes_disabled']
# 存入本地
f_name = str(a.year)+'-'+ length0(a.month)+'-'+ length0(a.day)+'-'+ length0(a.hour)+'-'+length0(a.minute)+'-'+length0(a.second)
dtation_info_df.to_csv('E:\\CitiBike\\y0_output\\' + f_name + '.csv')

In [ ]:
#-------------------------------------------  天气数据爬取（以后不必运行） --------------------------------------------------- 
from bs4 import BeautifulSoup
import requests
import csv
import bs4
 
def get_weather(url):      
    r = requests.get(url)
    r.raise_for_status()
    r.encoding = r.apparent_encoding
    soup = BeautifulSoup(r.text,'lxml')
    trs = soup.find_all('tr')#---tr
    for i in range(len(trs)):
        if 'Solar' in str(trs[i]):
            trs = trs[i+2:]
            break

    weather = pd.DataFrame(columns=['Time','Temperature','Dew Point','Humidity','Wind','Speed','Gust','Pressure','Precip_Rate','Precip_Accum','UV','Solar'])
    for tr in trs:
        if 'info' in locals().keys() and (tr.find('strong')).string[0:2] == info[0][0:2]:
            continue
        info = []
        count = 0
        for td in tr:
            if len(td.find_all('strong')) == 1:
                info.append(td.find_all('strong')[0].string)
            else:
                spans = td.find('span')
                for span in spans:
                    try:
                        info.append(float(span.string))
                        break
                    except:
                        pass
            if len(info) == count:
                info.append('none')
            count += 1
        weather = weather.append(pd.DataFrame([info],columns=['Time','Temperature','Dew Point','Humidity','Wind','Speed','Gust','Pressure','Precip_Rate','Precip_Accum','UV','Solar']),ignore_index = True)
    return weather

def get_weather2(date_list,get_station):
    df_weather = pd.DataFrame()
    miss_date = []
    for date in date_list:
        if date in list(work_per_hour['date']):
            url = 'https://www.wunderground.com/dashboard/pws/' + get_station +'/table/'+ date+"/" + date + "/daily"
            weather = get_weather(url)
            if len(weather) == 24:
                df_weather = pd.concat([df_weather,weather])
            else:
                print(date)
                miss_date.append(date)
    return miss_date,df_weather
#————————————————————---------------——调 用——————————————————————————————
#df_weather1
temp =  [datetime.datetime.strftime(x,'%Y-%m-%d') for x in list(pd.date_range(start='01/01/2021', end='05/31/2021'))]
[miss_data,df_weather1] = get_weather2(temp,'KNYNEWYO1267')

#df_weather2   此段时间内KNYNEWYO1267气象点停止工作，选取了其附近area0~19的20个气象点的数据
df_weather2 = pd.DataFrame()
for i in range(73):
    date = work_per_hour['date'][2160+i*24]
    temp = pd.read_csv("E:/CitiBike/Weather/"+date+"_area0.csv",index_col=0)#excel分表操作而得
    df_weather2 = pd.concat([df_weather2,temp])

#df_weather3
temp =  [datetime.datetime.strftime(x,'%Y-%m-%d') for x in list(pd.date_range(start='9/30/2021', end='31/01/2022'))]
[miss_data,df_weather3] = get_weather2(temp,'KNYNEWYO1267')

df_weather = pd.concat([df_weather1,df_weather2,df_weather3],ignore_index=True)

df_weather.to_csv('E:/CitiBike/y0_output/Weather.csv')